# Sylhet Floods 2022
## matchToFusion.ipynb
This script creates copies of the various flood map outputs, resampled and matched to the same resolution and pixel alignment as the Fusion model outputs, which are at 500 m resolution.

Fusion model outputs can be downloaded from Jonathan Giezendanner's GitHub: https://github.com/GieziJo/cvpr23-earthvision-CNN-LSTM-Inundation

For input rasters from Sentinel-1 (10 m resolution), the output is the average value in the 500 m pixel, thus the resuting raster is of fractional inundated area per pixel, with values ranging between 0 and 1.

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask
from datetime import datetime, timezone
import rioxarray as riox
import xarray

In [2]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data')

In [3]:
# Set the module path for helpers scripts
module_path = os.path.abspath(os.path.join('C:/Users/alexa/Documents/GitHub/Sylhet2022Floods/scr/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# Import module from analyse_modifiedDevries containing helpful functions to use
import importlib
import helpers.prepare_flood_raster as prep_raster
importlib.reload(prep_raster)

<module 'Helpers.prepare_flood_raster' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\Sylhet\\Helpers\\prepare_flood_raster.py'>

## PART 1: Define the fusion outputs for matching the other rasters to

In [5]:
# Get fusion files - these can be downloaded from Jonathan Giezendanner's GitHub
# https://github.com/GieziJo/cvpr23-earthvision-CNN-LSTM-Inundation
fusionPath = rootPath/'Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble'
fusionFiles = [file for file in list(fusionPath.iterdir()) if file.suffix=='.tiff']

# Define start and end of data we want for Sylhet
targetStart='20220501'
targetEnd='20220831'

# Get names of GFD files during Sylhet event
targetFusionFiles = [file for file in fusionFiles if datetime.fromtimestamp(int(file.stem[:-3])).strftime('%Y%m%d') >= targetStart
                 and datetime.fromtimestamp(int(file.stem[:-3])).strftime('%Y%m%d') <= '20220901']
targetFusionFiles

[WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1651708800000.tiff'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1652400000000.tiff'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1653091200000.tiff'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1653782400000.tiff'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1654473600000.tiff'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1655164800000.tiff'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1655856000000.tiff'),
 WindowsPath('Z:/media/mule

In [6]:
# get dates from the filenames
fusionDates = [datetime.fromtimestamp(int(file.stem[:-3])).strftime('%Y%m%d') for file in targetFusionFiles]
# fusionDates

In [7]:
# Define the target raster from whcih to match other rasters extent etc to
targetRaster = targetFusionFiles[0]

In [8]:
targetFusionFiles[0]

WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Models/DeepLearning/Inference/CrossValidation/Historical/Ensemble/1651708800000.tiff')

## PART 2: Match the GFD outputs
Create new rasters at same resolution and pixel alignment as fusion model outputs

In [19]:
# Crreate area for saving all "matched" rasters
matchedPath = rootPath/'Raster/SylhetMatchedToFusion'
matchedPath.mkdir(exist_ok=True)

In [17]:
outputPath=matchedPath/'GFD'
outputPath.mkdir(exist_ok=True)

In [18]:
# Get the files that need to be matched
GFDPath = rootPath/'Raster/ModisHistoricalGFD/WeeklyBangladeshBB'
GFDFiles = [file for file in list(GFDPath.iterdir()) if file.suffix=='.tif']

# get dates from the filenames
midDates = [datetime.fromtimestamp(int(file.stem.split('_')[0][:-3])).strftime('%Y%m%d') for file in GFDFiles]
sttDates = [datetime.fromtimestamp(int(file.stem.split('_')[1][:-3])).strftime('%Y%m%d') for file in GFDFiles]
endDates = [datetime.fromtimestamp(int(file.stem.split('_')[2][:-3])).strftime('%Y%m%d') for file in GFDFiles]

# Get names of GFD files during Sylhet event
targetGFDFiles = [file for file in GFDFiles if datetime.fromtimestamp(int(file.stem.split('_')[0][:-3])).strftime('%Y%m%d') >= targetStart
                 and datetime.fromtimestamp(int(file.stem.split('_')[1][:-3])).strftime('%Y%m%d') <= targetEnd]

In [26]:
# Loop through files and create new matched raster for each
for file in targetGFDFiles:
    date = datetime.fromtimestamp(int(file.stem.split('_')[1][:-3])).strftime('%Y%m%d') # mid date
    outputFile = outputPath/(date+'.tif')
    print('Creating:', outputFile)
    if os.path.exists(outputFile):
        continue
    prep_raster.matchWriteRaster(file, targetRaster, outputFile)

Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220430.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220508.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220516.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220524.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220601.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220609.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220617.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220625.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220703.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFD\20220711.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToF

## PART 3: Match the Thomas et al. outputs
Create new rasters at same resolution and pixel alignment as fusion model outputs

In [60]:
outputPath=matchedPath/'Devries'
outputPath.mkdir(exist_ok=True)

In [61]:
# Open the modified Devries mosaiced raster files and get the image dates
DevriesPath = rootPath/'Raster/Sylhet/Sen1MitchellSingleOrbit/Mosaic'
DevriesFiles = [file for file in (DevriesPath).iterdir() if file.is_file() and '2022' in str(file) and file.suffix=='.tif']

In [63]:
# Loop through files and create new matched raster for each
for file in DevriesFiles:
    date = file.stem[-14:-6]
    outputFile = outputPath/(date+'.tif')
    print('Creating:', outputFile)
    if os.path.exists(outputFile):
        continue
    prep_raster.matchWriteRaster(file, targetRaster, outputFile)

Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\Devries\20220501_t.tif


## PART 4: Match the NASA IMPACT outputs
Create new rasters at same resolution and pixel alignment as fusion model outputs

In [87]:
outputPath=matchedPath/'IMPACT'
outputPath.mkdir(exist_ok=True)

In [88]:
# Open the modified Devries mosaiced raster files and get the image dates
IMPACTPath = rootPath/'Raster/SylhetNASAImpact/5_Preds/Mosaic/'
IMPACTFiles = [file for file in (IMPACTPath).iterdir() if file.is_file() and '2022' in str(file) and file.suffix=='.tif']

In [89]:
# Loop through files and create new matched raster for each
for file in IMPACTFiles:
    date = file.stem
    outputFile = outputPath/(date+'.tif')
    print('Creating:', outputFile)
    if os.path.exists(outputFile):
        continue
    prep_raster.matchWriteRaster(file, targetRaster, outputFile)

Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220501.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220511.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220513.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220523.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220525.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220604.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220606.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220616.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220618.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\IMPACT\20220628.tif
Creating: Z:\media\mule\Projects\NASA\NI

## PART 5: Match the GFM outputs
Create new rasters at same resolution and pixel alignment as fusion model outputs

In [93]:
outputPath=matchedPath/'GFM'
outputPath.mkdir(exist_ok=True)

In [94]:
# Open the mosaiced raster files and get the image dates
GFMPath = rootPath/'Raster/SylhetCopernicusGFM/sylhet_district_2022-05-01-2022-08-31/Mosaic'
GFMFiles = [file for file in (GFMPath).iterdir() if file.is_file() and '2022' in str(file) and file.suffix=='.tif']

In [95]:
# Loop through files and create new matched raster for each
for file in GFMFiles:
    date = file.stem[-10:].replace('_','')
    outputFile = outputPath/(date+'.tif')
    print('Creating:', outputFile)
    if os.path.exists(outputFile):
        continue
    prep_raster.matchWriteRaster(file, targetRaster, outputFile)

Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220501.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220511.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220513.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220523.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220525.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220604.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220606.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220616.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220618.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\GFM\20220628.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToF

## PART 6: Match the UNOSAT outputs
Create new rasters at same resolution and pixel alignment as fusion model outputs

In [100]:
outputPath=matchedPath/'UNOSAT'
outputPath.mkdir(exist_ok=True)

In [101]:
# Open the mosaiced raster files and get the image dates
UNOSATPath = rootPath/'Raster/SylhetUNOSAT'
UNOSATFiles = [file for file in (UNOSATPath).iterdir() if file.is_file() and 'allTouchFalse' in str(file) and file.suffix=='.tif']
UNOSATFiles.reverse()
UNOSATDates = ['20220525','20220619']

In [102]:
UNOSATFiles

[WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetUNOSAT/S1_20220525_WaterExtent_NE_allTouchFalse.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetUNOSAT/RCM1_20220619_WaterExtent_NE_allTouchFalse.tif')]

In [103]:
# Loop through files and create new matched raster for each
for i, file in enumerate(UNOSATFiles):
    date = UNOSATDates[i]
    outputFile = outputPath/(date+'.tif')
    print('Creating:', outputFile)
    if os.path.exists(outputFile):
        continue
    prep_raster.matchWriteRaster(file, targetRaster, outputFile)

Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\UNOSAT\20220525.tif
Creating: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetMatchedToFusion\UNOSAT\20220619.tif
